# Notebook 4 - pandas
[pandas](http://pandas.pydata.org) provides high-level data structures and functions designed to make working with structured or tabular data fast, easy and expressive. The primary objects in pandas that we will be using are the `DataFrame`, a tabular, column-oriented data structure with both row and column labels, and the `Series`, a one-dimensional labeled array object.

pandas blends the high-performance, array-computing ideas of NumPy with the flexible data manipulation capabilities of spreadsheets and relational databases. It provides sophisticated indexing functionality to make it easy to reshape, slice and perform aggregations.

While pandas adopts many coding idioms from NumPy, the most significant difference is that pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast, is best suited for working with homogeneous numerical array data.
<br>

## Table of Contents:
- [Data Structures](#structures)
    - [Series](#series)
    - [DataFrame](#dataframe)
- [Essential Functionality](#ess_func)
    - [Removing Entries](#removing)
    - [Indexing](#indexing)
    - [Arithmetic Operations](#arithmetic)
- [Summarizing and Computing Descriptive Statistics](#sums)
- [Loading and storing data](#loading)
- [Data Cleaning and preperation](#cleaning)
    - [Handling missing data](#missing)
    - [Data transformation](#transformation)

The common pandas import statment is shown below:

In [ ]:
# Common pandas import statement
import numpy as np
import pandas as pd

# Data Structures <a name="structures"></a>
## Series <a name="series"></a>
A Series is a one-dimensional array-like object containing a sequence of values and an associated array of data labels called its index.

The easiest way to make a Series is from an array of data:

In [ ]:
data = pd.Series([4, 7, -5, 3])

Now try printing out data

In [ ]:
data

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Because we didn't specify an index, the default one is simply integers 0 through N-1.

It is possible to get only the index or only the values of a Serioes with:

In [ ]:
data.values

You can specify custom indices when intialising the Series

In [ ]:
data2 = pd.Series([4, 7, -5, 3], index=["a", "b", "c", "d"])

In [ ]:
data2

Another way to think about Series is as a fixed-length ordered dictionary. Furthermore, you can actually define a Series in a similar manner to a dictionary

In [ ]:
cities = {"Glasgow" : 599650, "Edinburgh" : 464990, "Aberdeen" : 196670, "Dundee" : 147710}
data3 = pd.Series(cities)

In [ ]:
data3

## DataFrame <a name="dataframe"></a>
A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type. The DataFrame has both row and column index and can be thought of as a dict of Series all sharing the same index.

The most common way to create a DataFrame is with dicts:

In [ ]:
data = {"cities" : ["Glasgow", "Edinburgh", "Aberdeen", "Dundee"],
        "population" : [599650, 464990, 196670, 147710],
        "year" : [2011, 2013, 2013, 2013]}
frame = pd.DataFrame(data)

Try printing it out

In [ ]:
frame

Jupyter Notebooks prints it out in a nice table but the basic version of this is also just as readable!

Additionally you can also specify the order of columns, or the row index,  during initialisation

In [ ]:
frame2 = pd.DataFrame(data, columns=["year", "cities", "population"], index=["a", "b", "c", "d"])
frame2

You can retrieve a particular column from a DataFrame with
```python
frame["cities"]
```
The result is going to be a Series.

In [ ]:
frame2["cities"]

It is also possible to add and modify the columns of a DataFrame

In [ ]:
frame2["size"] = 100
frame2

In [ ]:
frame2["size"] = [175, 264, 65.1, 60]  # in km^2
frame2

Here is a table of different ways of initialising a DataFrame for your reference

| Type | Notes |
| --- | --- |
| 2D ndarray | A matrix of data; passing optional row and column labels |
| dict of arrays, lists, or tuples | Each sequence becomes a column in the DataFrame; all sequences must be the same length |
| dict of Series | Each value becomes a column; indexes from each Series are unioned together to<br>form the result's row index if not explicit index is passed |
| dict of dicts | Each inner dict becomes a column; keys are unioned to form the row<br>index as in the "dict of Series" case |
| List of dicts or Series | Each item becomes a row in the DataFrame; union of dict keys or<br>Series indices becomes the DataFrame's column labels |
| List of lists or tuples | Treated as the "2D ndarray" case |

# Essential Functionality <a name="ess_func"></a>
In this section, we will go through the fundamental mechanics of interacting with the data contained in a Series or DaraFrame.

## Removing columns/indices <a name="removing"></a>
Similar to above, it is easy to remove entries. This is done with the `drop()` method and can be applied to both columns and indices:

In [ ]:
# define new DataFrame
data = np.reshape(np.arange(9), (3,3))
df = pd.DataFrame(data,
                  index=["a", "b", "c"],
                  columns=["Edinburgh", "Glasgow", "Aberdeen"])

df

In [ ]:
df.drop("b")  # remove row (index)

In [ ]:
# You can also drop from a column
df.drop(["Aberdeen", "Edinburgh"], axis="columns")

Note that the original data frame is unchanged: `df.drop()` gives us a new data frame with the desired data dropped, and leaves the original data intact. We can ask `.drop()` to operate directly on the original data frame by setting the argument `inplace=True`.

In [ ]:
df

## Indexing <a name="indexing"></a>

Indexing in pandas works simiarly to numpy, but we have to use `.iloc` instead of just normal indexing

In [ ]:
data = np.reshape(np.arange(9), (3,3))
df = pd.DataFrame(data, index=["a", "b", "c"],
                  columns=["Edinburgh", "Glasgow", "Aberdeen"])

df

In [ ]:
df[2]   # this won't work

In [ ]:
df.iloc[2]  # this works and return a series

You can also slice a dataframe as usual:

In [ ]:
df.iloc[:2]

You can index into many dimensions as seen in NumPy:

In [ ]:
df.iloc[2,0]

Finally, when the indices/column names of your data frame are not integers, you can use `.loc` to index:

In [ ]:
frame2

In [ ]:
frame2.loc["a", "size"]

## Arithmetic <a name="arithmetic"></a>
When you are performing arithmetic operations between two objects, if any index/column pairs are not the same, the respective indexes/columns in the result will be the union of the index pair. Let's have a look

In [ ]:
df1 = pd.DataFrame(np.arange(12).reshape((3,4)),
                  columns=list("abcd"))
df1

In [ ]:
df2 = pd.DataFrame(np.arange(12).reshape((3,4)),
                  columns=list("cdef"))
df2

In [ ]:
# adding the two
df1+df2

Notice how where we don't have matching values from `df1` and `df2` the output of the addition operation is `NaN` since there are no two numbers to add.

Well, we can "fix" that by filling in the `NaN` values. This effectively tells pandas where there are no two values to add, assume that the missing value is just zero.

In [ ]:
df1.add(df2, fill_value=0)

Here's a list of all arithmetic operations within pandas. They are similar to NumPy!

| Operator | Method | Description |
| -- | -- | -- |
| + | add | Addition |
| - | sub | Subtraction |
| / | div | Division |
| // | floordiv | Floor division |
| * | mul | Multiplication |
| ** | pow | Exponentiation |


# Summarizing and computing descriptive stats <a name="sums"></a>
`pandas` is equipped with common mathematical and statistical methods. Most of which fall into the category of reductions or summary statistics. These are methods that extract a single value from a list of values. For example, you can extract the sum of a `Series` object like this:

In [ ]:
df = pd.DataFrame(np.arange(20).reshape(5,4),
                 columns=["a", "b", "c", "d"])
df

In [ ]:
df.sum()

Notice how that created the sum of each column?

Well you can actually make that the other way around by adding an extra option to `sum()`

In [ ]:
df.sum(axis="columns")

A similar method also exists for obtaining the mean of data:

In [ ]:
df.mean()

Finally, the mother of the methods we discussed here is `describe()` 

In [ ]:
df.describe()

Here are some of the summary methods:

| Method | Description |
| -- | -- |
| count          | Return number of non-NA values |
| describe       | Set of summary statistics |
| min, max       | Minimum, maximum values |
| argmin, argmax | Index locations at which the minimum or maximum value is obtained | 
| sum            | Sum of values |
| mean           | Mean of values |
| median         | Arithmetic median of values |
| std            | Sample standard deviation of values
| value_counts() | Counts the number of occurrences of each unique element in a column |

### Exercise 1
A dataset of random numbers is created below. Obtain all rows starting from row 85 to 97.

*Note: Remember that Python uses 0-based indexing*

In [ ]:
df = pd.DataFrame(np.reshape(np.arange(10000), (100,100)))


### Exercise 2
Create a (3,3) DataFrame and square all elements in it.

### Exercise 3

A random DataFrame is created below. Find the median value of each column.

In [ ]:
df = pd.DataFrame(np.random.uniform(0, 10, (100, 100)))


# Data Loading and Storing <a name="loading"></a>
Accessing data is a necessary first step for data science.

A very common format is the `.csv`. This is an easy to read file format which is usually visualised like a spreadsheet. The data itself is usually separated with a `,` which is called the **delimiter**.

Here is an example of a `.csv` file:
```
name,sex
Joseph,M	
Andrew,M	
Joshua,M	
Abigail,F	
Benjamin,M	
Anthony,M	
James,M	
Emily,F	
Elijah,M	
Matthew,M	
Daniel,M	
Aiden,M	
Alexander,M	
Ava,F	
Jayden,M	
Michael,M	
Liam,M	
William,M	
Olivia,F	
Noah,M	
```

It details some names and sexes of newborn babies. The first line is called the header, and you can imagine that it is the name of the columns of a spreadsheet.

Let's now see how we can load this data and analyse it. The file is located in the folder `data` and is called `yob2012.txt`. We can read it like this:

In [ ]:
babes = pd.read_csv("data/yob2012.csv")
babes

Easy right?

### Exercise 4
Load the file `data/homes.csv` and find the mean selling price of these houses.

The `read_csv` function has a lot of optional arguments (more than 50). It's impossible to memorise all of them - it's usually best just to look up the particular functionality when you need it. 

You can search `pandas read_csv` online and find all of the documentation.

There are also many other functions that can read textual data. Here are some of them:

| Function | Description
| -- | -- |
| read_csv       | Load delimited data from a file, URL, or file-like object. The default delimiter is a comma `,` |
| read_table     | Load delimited data from a file, URL, or file-like object. The default delimiter is tab `\t` |
| read_clipboard | Reads the last object you have copied (Ctrl-C) |
| read_excel     | Read tabular data from Excel XLS or XLSX file |
| read_hdf       | Read HDF5 file written by pandas |
| read_html      | Read all tables found in the given HTML document |
| read_json      | Read data from a JSON string representation |
| read_sql       | Read the results of a SQL query |

*Note: there are also other loading functions which are not touched upon here*

# Data Cleaning <a name="cleaning"></a>
While doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming and rearranging. Such tasks are often reported to take **up to 80%** or more of a data analyst's time. Often the way the data is stored in files isn't in the correct format and needs to be modified. Researchers usually do this on an ad-hoc basis using programming languages like Python.

In this chapter, we will discuss tools for handling missing data, duplicate data, string manipulation, and some other analytical data transformations.

## Handling missing data <a name="missing"></a>
Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible.

In pandas, missing numeric data is represented by `NaN` (Not a Number) and can easily be handled:

In [ ]:
string_data = pd.Series(['orange', 'tomato', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull()

The pandas `NaN` is functionally equlevant to the standard Python type `NoneType` which can be defined with `x = None`.

Here are some other methods which you can find useful:
    
| Method | Description |
| -- | -- |
| dropna | Filter axis labels based on whether the values of each label have missing data|
| fillna | Fill in missing data with some value |
| isnull | Return boolean values indicating which values are missing |
| notnull | Negation of isnull |

Just like `.drop()`, these methods all return a new object, leaving the original unchanged (this behaviour can be overridden using the argument `inplace=True`).

### Exercise 5
Remove the missing data below using the appropriate method

In [ ]:
data = pd.Series([1, None, 3, 4, None, 6])
data

`dropna()` by default removes any row/column that has a missing value. What if we want to remove only rows in which all of the data is missing though?

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., None, None],
                    [None, None, None], [None, 6.5, 3.]])
data

In [ ]:
data.dropna()

In [ ]:
data.dropna(how="all")

### Exercise 6
That's fine if we want to remove missing data, what if we want to fill in missing data? Do you know of a way? Try to fill in all of the missing values from the data below with **0s**

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [2., None, None],
                    [None, None, None], [None, 1.5, 9.]])
data

## Data Transformation  <a name="transformation"></a>
### Removing duplicates
Duplicate data can be a serious issue, luckily pandas offers a simple way to remove duplicates

In [ ]:
data = pd.DataFrame([1, 2, 3, 4, 3, 2, 1])
data

In [ ]:
data.drop_duplicates()

You can also select which rows to keep

In [ ]:
data.drop_duplicates(keep="last")

### Replacing data
You've already seen how you can fill in missing data with `fillna()`. That is actually a special case of more general value replacement. That is done via the `replace()` method.

Let's consider an example where the dataset given to us had `-999` as sentinel values for missing data instead of `NaN`.

In [ ]:
data = pd.DataFrame([1., -999., 2., -999., 3., 4., -999, -999, 7.])
data

In [ ]:
data.replace(-999, np.nan)

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

Suppose you now want to lower all values exceeding 2 from one of the columns.

In [ ]:
col = data[2]
col[col > 2]

We could set a ceiling for these values at 2.

In [ ]:
data[data > 2] = 2
data.describe()

What we just did is called **boolean indexing**.

### Exercise 7
Let's load again our file with home prices and filter out homes based on our preference:
1. Load up the file `data/homes.csv`
2. The data contains some duplicates. Filter them out.
3. Let's say that the most we can spend on a house is £150. Keep only houses that have a **sell**ing price less than £150 and remove the rest
4. Select only houses that have 4 or more bedrooms
5. Select only houses that have 3 or more baths

You should end up with only 2 houses

## Further Resources
We have now reached the end of the core notebooks for this course, covering the essential tools for any data science project using Python. There are now a range of extension notebooks introducing more specific topics which we encourage you to explore. these are:
- `python-data-extra-machine-learning`, introducing machine learning with scikit-learn.
- `python-data-extra-networks`, introducing network analysis with NetworkX.
- `python-data-extra-regex`, which introdices regular expressions, a powerful tool for working with text data.
- `python-data-extra-text-analysis`, introducing tools for analysing text and performing sentiment analysis (see also `python-data-extra-regex`)
- `python-data-extra-scipy`, introducing SciPy and exploring its signal processing module.

Additonally, there is a notebook `python-data-extra-life` which is on John Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life). This is not directly related to data science, but is a fun exercise in programming if you want to sharpen your skills in this area. This notebook was created by Alisdair Tullo, and is reproduced here with his kind permission.

The book [Python for Data Analysis](https://discovered.ed.ac.uk/primo-explore/fulldisplay?docid=44UOE_ALMA51179646230002466&context=L&vid=44UOE_VU2&lang=en_US&search_scope=default_scope&adaptor=Local%20Search%20Engine&isFrbr=true&tab=default_tab&query=any,contains,python%20data%20analysis&sortby=date&facet=frbrgroupid,include,1619270935&offset=0) by Wes McKinney is avaliable from the library. It provides a more comprehensive overview of data science in Python, and in particular the later chapters extend and build on what has been developed so far in this course. Further, there are a host of Lynda.com courses available to extend your skills. [Python for Data Science Essential Training](https://www.lynda.com/Python-tutorials/Python-Data-Science-Essential-Training/520233-2.html) begins by crossing over with the core notebooks of this course, and goes on to introduce intereting topics including maths and stats, dimensionality reduction, outlier analysis, Bayesian analysis, and web scraping. [Python for Data Science Tips, Tricks & Techniques]() introduces methods for reading in data in arange of formats, further techniques for exploring data using pandas, and an alternative library called `ggplot` for plotting data.